In [2]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# Load the data
file_path = 'data.csv'
df = pd.read_csv(file_path)

df.head()

,category,subcategory,right-right,wrong-wrong,right-wrong,wrong-right
0,Short_question,trivia,25,0,0,0
1,Computational,mathmatics,5,5,15,0
2,Computational,statistics,17,3,4,1
3,Computational,machine_Learning,8,6,10,1
4,Computational,Social_science,14,3,8,0


# McNemar's test

## Overall

In [3]:
# Sum the discordant counts across all subcategories
total_right_wrong = df['right-wrong'].sum()
total_wrong_right = df['wrong-right'].sum()

# Create the 2x2 table for total data
total_table = [[0, total_right_wrong],
               [total_wrong_right, 0]]

# Perform McNemar's test on the total table
total_result = mcnemar(total_table, exact=False, correction=True)

# Output the result
print(f"Right-Wrong (Neutral correct, Biased incorrect): {total_right_wrong}")
print(f"Wrong-Right (Neutral incorrect, Biased correct): {total_wrong_right}")
print(f"McNemar's test p-value: {total_result.pvalue}")

Right-Wrong (Neutral correct, Biased incorrect): 37
Wrong-Right (Neutral incorrect, Biased correct): 2
McNemar's test p-value: 5.1992785565346994e-08


## For Each Subject

In [4]:
# Function to perform McNemar's test
def run_mcnemar_test(row):
    if row['right-wrong'] == 0 and row['wrong-right'] == 0:
        return float('nan')  # Handle cases where both counts are zero
    table = [[0, row['right-wrong']],
             [row['wrong-right'], 0]] # Create the contingency table
    result = mcnemar(table, exact=False, correction=True)
    return result.pvalue

# Apply the test to each row
df['mcnemar_pvalue'] = df.apply(run_mcnemar_test, axis=1)

# Show results
result_df = df[['subcategory', 'right-wrong', 'wrong-right', 'mcnemar_pvalue']]
result_df


,subcategory,right-wrong,wrong-right,mcnemar_pvalue
0,trivia,0,0,NaN
1,mathmatics,15,0,0.000301
2,statistics,4,1,0.371093
3,machine_Learning,10,1,0.015861
4,Social_science,8,0,0.013328


In [11]:
pvals = list(result_df['mcnemar_pvalue'].values)[1:]
pvals

[0.00030059760744045053,
 0.37109336952269756,
 0.015861332739773026,
 0.01332832878081758]

In [58]:
q = 0.05
m = 4
for i, p in enumerate(sorted(pvals)):
    significance = (i+1)/m * q
   
    
    if significance > p:
        print("reject ---", round(significance,3), ">", round(p,5))
    else:
        print("accept ---", round(significance,3), "<", round(p,3))

reject --- 0.013 > 0.0003
reject --- 0.025 > 0.01333
reject --- 0.038 > 0.01586
accept --- 0.05 < 0.371


In [21]:
sorted(pvals)

[0.00030059760744045053,
 0.01332832878081758,
 0.015861332739773026,
 0.37109336952269756]